In [1]:
import os
from dotenv import load_dotenv
from pyprojroot import here
from langchain.chains import create_sql_query_chain
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

load_dotenv()

True

**Set the environment variable and load the LLM**

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")


llm = ChatOpenAI(model="gpt-4", temperature=0)
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# llm = ChatOpenAI(model="gpt-4o")

**Load and test the sqlite db**

In [3]:
db_path = str(here("data")) + "/ICDD-energy.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(db)

# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Comments LIMIT 10;")

sqlite
['Applications', 'Comments', 'Compositions', 'Melting Points']


"[(353, 10359, None, None, 'Decomposes at 1750 C'), (494, 10503, None, None, None), (569, 10579, None, None, None), (618, 10628, None, None, 'Transition to calcite at 520. Antacid'), (636, 10646, None, 'A new method of X-Ray crystal analysis', None), (667, 10677, None, None, None), (696, 10706, None, None, None), (738, 10750, None, None, None), (780, 10792, None, None, None), (784, 10796, None, None, None)]"

**Create the SQL agent and run a test query**

In [4]:
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many rows are there in the Compositions table?"})
response

'SELECT COUNT(*) FROM "Compositions";'

In [5]:
db.run(response)

'[(19695,)]'